In [ ]:
import pandas as pd
import numpy as np
import math as mt
import datetime
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Importing input files and cleaning them
esb_name = input("Please choose one ESB from the following list: ars, csu, dv, enmom, groh, rv \n ")
xls = pd.ExcelFile(esb_name + "_positions_returns_5y.xlsx")
stock_returns = pd.read_csv("stock_returns_19y.csv", header = None)
stock_returns = stock_returns.rename({0:'Date', 1:'Tickers',3:'Returns'}, axis = 1)
stock_returns['Returns'] = stock_returns['Returns'].fillna(0)
spx_returns = pd.read_csv("SPX_returns.csv", header = None)
spx_returns = spx_returns.rename({0:'Date', 1:'Returns'}, axis = 1)
listlong_full = xls.parse('Stocks List - Long', header = None)
listlong_full = listlong_full.rename({0:'Date', 1:'Tickers'}, axis = 1)

listshort_full = xls.parse('Stocks List - Short', header = None)
listshort_full = listshort_full.rename({0:'Date', 1:'Tickers'}, axis = 1)

retlong = xls.parse('Return - Long', header = None)
retlong = retlong.rename({0:'Date', 1:'Returns'}, axis = 1)
retlong['Returns'] = retlong['Returns'].fillna(0)

retshort = xls.parse('Return - Short', header = None)
retshort = retshort.rename({0:'Date', 1:'Returns'}, axis = 1)

ret = xls.parse('Total Return', header = None)
ret = ret.rename({0:'Date', 1:'Returns'}, axis = 1)

dates_full = pd.to_datetime(ret['Date'])

In [ ]:
# Filling NA in returns data with 0
stock_returns['Returns'] = stock_returns['Returns'].fillna(0)

In [ ]:
# Gather date input from user
usr = input("Please choose of the following numbers:\n \
        1. Verify the portfolio return for a single day based on positions in the 5y hist from December 2006 - November 2011.\n \
        2. Verify the portfolio return for a range of dates based on positions in the 5y hist from December 2006 - November 2011.\n")
if usr == '1':
    date = input('Please enter the date in an dd/mm/yyyy format between December 2006 - November 2011.')
    index_long = listlong_full[listlong_full['Date'] == date].index[0]
    index_short = listshort_full[listshort_full['Date'] == date].index[0]
    listlong = listlong_full['Tickers'][listlong_full['Date'] == date]
    listshort = listshort_full['Tickers'][listshort_full['Date'] == date]
    if([stock_returns['Date'] == date][0].any()):

        stock_returns_day = stock_returns[stock_returns['Date'] == date]
        listlong = listlong.str.split("'", expand = True)
        listlong.columns.values[0] = 0
        listlong = listlong[listlong != ' ']
        listlong = listlong[listlong != '[']
        listlong = listlong[listlong != ']']
        listlong = listlong[listlong != "\n "]                    
        listlong = listlong.T.dropna()

        listshort = listshort.str.split("'", expand = True)
        listshort.columns.values[0] = 0
        listshort = listshort[listshort != ' ']
        listshort = listshort[listshort != '[']
        listshort = listshort[listshort != ']']
        listshort = listshort[listshort != "\n "]                    
        listshort = listshort.T.dropna()

        listlong = listlong[index_long].tolist()
        listshort = listshort[index_short].tolist()

        #Listing out top 5 and bottom 5 percentile tickers for the Factor Bucket portfolio
        print("\n\nTop and Bottom 5-Percentile Tickers:\n\n")
        print("Long Tickers:\n\n", listlong,"\n\n")
        print("Short Tickers:\n\n", listshort,"\n\n")

        long = stock_returns_day['Returns'][stock_returns_day['Tickers'].isin(listlong)].mean()
        short = stock_returns_day['Returns'][stock_returns_day['Tickers'].isin(listshort)].mean()
        totret = long - short # Final Returns = Long Return Average - Short Return Average

        print("Portfolio Stats: \n\n")
        print("Calculated Portfolio Returns for the requested date:",round(totret,2))
        print("QMIT Reported Portfolio returns for the requested date:",round(ret.loc[ret[ret['Date'] == date].index[0] + 1]['Returns'],2))
    else:
        print("\n\nRequested date not in Range. Please try again with another date.")
elif(usr == '2'):
    avg = []
    reported_ret = []
    returns_dates = []
    final = pd.DataFrame(columns = ['Date', 'Calculated Returns','QMIT Reported Returns'])
    dt = input('Please enter the start date in an dd/mm/yyyy format within December 2006 - November 2011.\n')
    dt1 = input('Please enter the end date in an dd/mm/yyyy format within December 2006 - November 2011.\n')
    date = pd.to_datetime(dt, format="%d/%m/%Y")
    date1 = pd.to_datetime(dt1, format="%d/%m/%Y")
    stock_returns['Date'] = pd.to_datetime(stock_returns['Date'], format="%d/%m/%Y")
    ret['Date'] = pd.to_datetime(ret['Date'], format="%d/%m/%Y")
    if(date>date1):
        print("Invalid Dates: End date is before the Start date. Please re-run the cell.")
    else:
        dates = stock_returns['Date'][(stock_returns['Date']>=date) & (stock_returns['Date']<=date1)].unique()
        for day in dates:
            index_long = listlong_full[listlong_full['Date'] == day].index[0]
            index_short = listshort_full[listshort_full['Date'] == day].index[0]
            listlong = listlong_full['Tickers'][listlong_full['Date'] == day]
            listshort = listshort_full['Tickers'][listshort_full['Date'] == day]
            if([stock_returns['Date'] == day][0].any()):
                stock_returns_day = stock_returns[stock_returns['Date'] == day]
                listlong = listlong.str.split("'", expand = True)
                listlong.columns.values[0] = 0
                listlong = listlong[listlong != ' ']
                listlong = listlong[listlong != '[']
                listlong = listlong[listlong != ']']
                listlong = listlong[listlong != "\n "]                    
                listlong = listlong.T.dropna()

                listshort = listshort.str.split("'", expand = True)
                listshort.columns.values[0] = 0
                listshort = listshort[listshort != ' ']
                listshort = listshort[listshort != '[']
                listshort = listshort[listshort != ']']
                listshort = listshort[listshort != "\n "]                    
                listshort = listshort.T.dropna()

                listlong = listlong[index_long].tolist()
                listshort = listshort[index_short].tolist()

                long = stock_returns_day['Returns'][stock_returns_day['Tickers'].isin(listlong)].mean()
                short = stock_returns_day['Returns'][stock_returns_day['Tickers'].isin(listshort)].mean()
                totret = long - short
                avg.append(totret)
                reported_ret.append(ret.loc[ret[ret['Date'] == day].index[0] + 1]['Returns'])
                returns_dates.append(ret.loc[ret[ret['Date'] == day].index[0] + 1]['Date']) 
        final.loc[:,'Date'] = pd.to_datetime(returns_dates)
        final.loc[:,'Calculated Returns'] = np.round(avg,2)
        final.loc[:,'QMIT Reported Returns'] = np.round(reported_ret,2)
        print(final.to_string())
else:
    print('The number chosen is invalid. Please re-run the cell and select either 1 or 2.')  

#### Performance Metrics 

In [ ]:
# Replicating the Annualized Return for full 19+ yr history 
ret['Date'] = pd.to_datetime(ret['Date'])
number_of_years = (dates_full.iloc[-1]-dates_full.iloc[0]).days/365 
ret = ret[ret['Date'].dt.year >= 2000] # returns data are available from January 2000
temp = []
for i in range(len(ret)):
    temp.append((1+ret['Returns'].iloc[i]/100))
annualized_ret = (np.prod(temp)**(1/number_of_years)-1)*100
print("Annualized Return:", round(annualized_ret,2))

In [ ]:
# Replicating the Annualized Volatility & Downside Deviation
vol = pd.DataFrame(columns  = ['Value'])
# for i in range (len(returns)):
vol.loc['Vol_B', 'Value'] = ret['Returns'].std()
vol.loc['Annualized Vol_B','Value'] = vol.loc['Vol_B','Value']*mt.sqrt(12)
vol.loc['Downside Dev','Value'] = np.sqrt(np.sum(ret[ret.loc[:,'Returns']<0].loc[:,'Returns'].values**2)/len(ret))
vol.loc['Ann Downside Dev','Value'] = vol.loc['Downside Dev','Value']*np.sqrt(12)
print("Annualized LTD Vol:", round(vol.loc['Annualized Vol_B','Value'],2))

In [ ]:
#Sharpe
sharpe = annualized_ret/vol.loc['Annualized Vol_B','Value']
print("Sharpe:", round(sharpe,2))

In [ ]:
#Sortino
sortino = annualized_ret/vol.loc['Ann Downside Dev','Value']
print("Sortino:", round(sortino,2))

In [ ]:
#Beta to S&P500
betasp = np.cov(ret['Returns'], spx_returns['Returns'].drop(index=0))[0][1]/np.cov(ret['Returns'], spx_returns['Returns'].drop(index=0))[1][1]
print("Beta to S&P500:", round(betasp,2))